# Série V
Nonlinear Time Series Analysis - Computer Science Master's Degree Course - Federal University of São Paulo (UNIFESP) - Prof. Elbert E. N. Macau

Author: Rafael Leiniö


## Algorithm Implementations

- [Henon Map](https://github.com/rafaelleinio/ntsa/blob/master/ntsa/algorithms/maps/henon.py)
- [Lorenz Map](https://github.com/rafaelleinio/ntsa/blob/master/ntsa/algorithms/maps/lorenz.py)
- [Rössler Map](https://github.com/rafaelleinio/ntsa/blob/master/ntsa/algorithms/maps/rossler.py)
- [Lyapunov Exponents](https://github.com/rafaelleinio/ntsa/blob/master/ntsa/algorithms/maps/lyapunov_exponents.py)

In [1]:
# fix working dir
import pathlib
import os
path = os.path.join(pathlib.Path().absolute(), '../..')
os.chdir(path)

In [2]:
# Lyapunov Exponents calculator

from numpy import *
from numpy.linalg import *

from ntsa.algorithms.maps.map import Map


class LyapunovExponents:
    def __init__(self, map: Map, tolerance: float = 0.00001, max_iterations: int = 1000):
        self.map = map
        self.tolerance = tolerance
        self.max_iterations = max_iterations

    def calculate_from_initial_conditions(self, initial_conditions: ndarray):
        # setup variables
        n = len(initial_conditions)
        x = initial_conditions
        w = eye(n)
        h = zeros(n)
        l = -1

        # Numerical Lyapunov exponents calculation
        for i in range(0, self.max_iterations):
            x_next, w_next = self.map.f(x), self.map.df(x, w)
            w_next = self._orthogonalize_columns(w_next)

            h_next = h + self._log_of_the_norm_of_the_columns(w_next)
            l_next = h_next / (i + 1)

            if norm(l_next - l) < self.tolerance:
                return sort(l_next)

            h = h_next
            x = x_next
            w = self._normalize_columns(w_next)
            l = l_next

        # Max iter reach without a Solution in desired tolerance
        raise ValueError(
            "Lyapunov Exponents calculation did not converge."
            f" Initial conditions = {initial_conditions}"
            f", Tolerance = {self.tolerance}"
            f", Max Iterations = {self.max_iterations}"
        )

    def calculate_over_set_of_initial_conditions(self, set: ndarray):
        solutions = array(
            [
                self.calculate_from_initial_conditions(initial_conditions)
                for initial_conditions in set
            ]
        )
        return apply_along_axis(lambda v: mean(v), 0, solutions)

    def _orthogonalize_columns(self, a):
        q, r = qr(a)
        return q @ diag(r.diagonal())

    def _normalize_columns(self, a):
        return apply_along_axis(lambda v: v / norm(v), 0, a)

    def _log_of_the_norm_of_the_columns(self, a):
        return apply_along_axis(lambda v: log(norm(v)), 0, a)


## Henon

In [3]:
# calculating Lyapunov Exponents for Henon map, with a=1.4, b=0.3
# with starting x=-0.5, y=0.3

from ntsa.algorithms.maps import Henon

henon = Henon(a=1.4, b=0.3)

starting_point = [-0.5, 0.25]

solver = LyapunovExponents(henon, tolerance=0.0001, max_iterations=10000)

In [4]:
solver.calculate_from_initial_conditions(starting_point)

array([-1.58552273,  0.38154993])

## Lorenz

In [5]:
# calculating Lyapunov Exponents for Lorenz map, with sigma=10, rho=28 and beta=8/3
# with starting x=0, y=2, z=0

from ntsa.algorithms.maps import Lorenz

lorenz_map = Lorenz(sigma=10, rho=28, beta=8 / 3)

starting_point = [0, 2, 0]

solver = LyapunovExponents(lorenz_map, tolerance=0.0001, max_iterations=10000)

In [6]:
solver.calculate_from_initial_conditions(starting_point)

array([-1.45699626e+01,  2.04232901e-03,  9.01254207e-01])

## Rössler

In [7]:
# calculating Lyapunov Exponents for Rössler map, with a=0.1, b=0.1 and c=14
# with starting x=0, y=2, z=1

from ntsa.algorithms.maps import Rossler

rossler_map = Rossler(a=0.1, b=0.1, c=14)

starting_point = [0, 2, 1]

solver = LyapunovExponents(rossler_map, tolerance=0.0001, max_iterations=10000)

In [8]:
solver.calculate_from_initial_conditions(starting_point)

array([ 0.06611168,  0.07243824, 14.05875559])